## Szöveg Generálás a Twilight c. könyv alapján 

# Feladat rövid leírása

A feladat célja, hogy egy új jelenetet készítsünk Stephanie Meyer Twilight című könyvébe. 
Ehhez Az első 5 fejezet első 3-4 oldalnyi szövegeit használjuk. 

## Készítette:

# Gellén Rebeka - J97N2Y

#### Szöveg beolvasása:

Ebben az első lépésben fogjuk a szövegünket és csupa kisbetűre állítjuk, megszüntetve a különbséget kis és nagybetűk között.

Ezután kiíratjuk a hosszát, illetve a szöveget teljes hosszában, tabulátorok, sortörések átalakításával.


In [1]:
with open("book1_script.txt", "r", encoding="utf8") as book:
    script = book.read().lower()

#a len() függvény fogja megszámolni a szövegünkben szereplő karakterek számát
print(len(script)) 

660165


In [1]:
#script[:660165] - ezzel megjeleníthetjük a teljes szöveget. A tabulátorok, sortörések is karakterekké alakulnak.
#Most ezt elrejtettem, mivel hosszú szöveggel dolgozunk és csak a helyet fogná.

#### felhasznált karakterek kigyűjtése a szövegből:

numpy: numerikus python - matematikai függvénykönyvtárral támogatja a többdimenziós tömbök és mátrixok használatát

tqdm: folyamat vizualizálására alkalmas

In [3]:
import numpy as np
from tqdm import tqdm

maxlen = 60
step = 3
sentences = []
next_chars = []

for i in tqdm(range(0, len(script) - maxlen, step)):
    sentences.append(script[i: i + maxlen])
    next_chars.append(script[i + maxlen])
len(sentences)

100%|██████████| 220035/220035 [00:00<00:00, 666722.30it/s]


220035

felhasznált karakterek

In [4]:
chars = sorted(list(set(script)))
print(chars)

['\t', '\n', ' ', '!', '"', "'", ',', '-', '.', '0', '1', '3', '4', '5', '6', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é', '\u2003', '—', '…']


A felhasznált karakterek indexei

In [5]:
char_indices = dict((char, chars.index(char)) for char in chars)
print(char_indices)


{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, "'": 5, ',': 6, '-': 7, '.': 8, '0': 9, '1': 10, '3': 11, '4': 12, '5': 13, '6': 14, '8': 15, '9': 16, ':': 17, ';': 18, '?': 19, 'a': 20, 'b': 21, 'c': 22, 'd': 23, 'e': 24, 'f': 25, 'g': 26, 'h': 27, 'i': 28, 'j': 29, 'k': 30, 'l': 31, 'm': 32, 'n': 33, 'o': 34, 'p': 35, 'q': 36, 'r': 37, 's': 38, 't': 39, 'u': 40, 'v': 41, 'w': 42, 'x': 43, 'y': 44, 'z': 45, 'é': 46, '\u2003': 47, '—': 48, '…': 49}


Ennél a résznél a bool-ra hibaüzenet érkezik, ami annyit kér, vagy használjunk szimpla bool-t, vagy np.bool_-t, ezutóbbi pedig nem változtat semmit, csupán elrejti a hibaüzenetet. Jobbnak láttam ezt választani.

In [6]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in tqdm(enumerate(sentences)):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print("x", x.shape)
print("y", y.shape)

220035it [00:09, 23226.43it/s]

x (220035, 60, 50)
y (220035, 50)


#### A model betanulása: 

A tensorflow modellek betanítására, alkalmazására jó. Képfeldolgozásnál is nagy haszna van, miután NLP-ről láttam, az oda való beadandómban is használtam.

In [7]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.GRU(16, input_shape=(maxlen, len(chars))),
    tf.keras.layers.Dense(units=len(chars), activation='softmax')
]) 

model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 16)                3264      
                                                                 
 dense (Dense)               (None, 50)                850       
                                                                 
Total params: 4,114
Trainable params: 4,114
Non-trainable params: 0
_________________________________________________________________


Itt növelem a számát az epochs-nak, hogy többször is feldolgozásra kerüljön a szöveg, hátha kevésbé nézne ki démonidézős szövegnek.

In [8]:
with tf.device("/CPU:0"):
    model.fit(x, 
            y,
            batch_size=128,
            epochs=10)

Epoch 1/10
1720/1720 [==============================] - 64s 33ms/step - loss: 2.1183
Epoch 2/10
1720/1720 [==============================] - 59s 34ms/step - loss: 1.9077
Epoch 3/10
1720/1720 [==============================] - 56s 33ms/step - loss: 1.8721
Epoch 4/10
1720/1720 [==============================] - 56s 33ms/step - loss: 1.8551
Epoch 5/10
1720/1720 [==============================] - 56s 33ms/step - loss: 1.8434
Epoch 6/10
1720/1720 [==============================] - 57s 33ms/step - loss: 1.8373
Epoch 7/10
1720/1720 [==============================] - 56s 33ms/step - loss: 1.8318
Epoch 8/10
1720/1720 [==============================] - 56s 33ms/step - loss: 1.8253
Epoch 9/10
1720/1720 [==============================] - 56s 33ms/step - loss: 1.8209
Epoch 10/10
1720/1720 [==============================] - 56s 33ms/step - loss: 1.8188


Kiindulás képpen megfogunk egy random sort. A szöveggenerálás során ez a sor fog kezdeni minden alkalommal.

In [9]:
import random

start_index = random.randint(0, len(script) - maxlen - 1)
base_text = script[start_index: start_index + maxlen]
base_text

' promised.  \nwe walked up the massive staircase, my hand tra'

In [10]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#### A szöveg elénk kerül: 

Miután megadtuk, hogy milyen hosszú szöveget szeretnénk, hogy generáljon nekünk, karakterenként íródik elénk.


In [11]:
import sys
temperatures = [0.2, 0.5, 1.0, 1.2] #0.2-esnél adta vissza a legolvashatóbb eredményt, 1.2-nél összefüggő szavakat sem tudott alkotni
generated_characters = 200

for temp in temperatures:
    print("Temp: ", temp)
    generated_text = base_text
    print(generated_text)
    for i in range(generated_characters):
        sampled = np.zeros((1, maxlen, len(chars)))    
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.          
        
        preds = model.predict(sampled, verbose=0)[0]      
        
        next_index = sample(preds, temp)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()

    print()

Temp:  0.2
 promised.  
we walked up the massive staircase, my hand tra
ce the was sace the gas and the tone the was was and my stared and the gas i greally the had was to the was to the gas i was was to the was his his stared and the was the was be on the was bee the was
Temp:  0.5
 promised.  
we walked up the massive staircase, my hand tra
ce somealt and the a noon the healnated on the back could hough the cack with my and to the don't the siced eyes stom in and but and the srin and my lough has an on she toncered it where. i was his ou
Temp:  1.0
 promised.  
we walked up the massive staircase, my hand tra
nten. he to tion."  
"theblly sid at me be in to the awe tor he his that ankne.  
es — i nousho. "and her me, that rearn.  
my was stizemy o stage. the os not-me at, calks. thly then' romed has doowcu
Temp:  1.2
 promised.  
we walked up the massive staircase, my hand tra
re."  
"i'ndward has caltny hive i wel uld cally tuld waving werew mown's, he how, cheet you lebpsly unsrime 

A kapott generált szöveg teljes nonsense, viszont ha még feljebb venném az egyes értékeket, a futási idő borzasztó hosszú lenne. (Őszintén, így is visszaadja, mennyire értelmetlen az eredeti szöveg.) Ez a sims-esített Alkonyat :D
